In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re

In [ ]:
# parse data to DataFrame

data_arr = []
with open('../input/web-server-access-logs/access.log') as log:
    for line in log.readlines():
        match = re.match(r'(.+) - - \[(\d{2}\/\w{3}\/\d{4}:\d{2}:\d{2}:\d{2}) \+\d{4}\] \"(\w+) .+', line)
        if match:
            data_arr.append([match.group(1), match.group(2), match.group(3)])

data = pd.DataFrame(data_arr, columns=['ip_address', 'datetime', 'request_type'])

In [ ]:
#Convert string datetime to Timestamp

data['datetime'] = data.datetime.apply(lambda x: x.replace('Jan', '01'))
data['datetime'] = pd.to_datetime(data.datetime, format='%d/%m/%Y:%H:%M:%S')

In [ ]:
#add datetime columns to group by

data['day'] = data['datetime'].apply(lambda x: x.day)
data['hour'] = data['datetime'].apply(lambda x: x.hour)
data['minute'] = data['datetime'].apply(lambda x: x.minute)

In [ ]:
#aggregate by minute and by hour

aggByMin = data.groupby(['day', 'hour', 'minute'], as_index=False).agg('count')\
  .rename({'ip_address': 'count'}, axis=1).drop(['datetime', 'request_type'], axis=1)

aggByHour = data.groupby(['day', 'hour'], as_index=False).agg('count')\
  .rename({'ip_address': 'count'}, axis=1).drop(['datetime', 'request_type', 'minute'], axis=1)

In [ ]:
#plot it

plt.plot(aggByMin['count'], label='load by minute')
plt.legend()


In [ ]:
plt.plot(aggByHour['count'], label='load by hour')
plt.legend()